<!-- Notebook Header Start -->

<h1 align="center">Spike Sorting</h1>

<p align="center">
  <strong>Author:</strong> Karl Bates<br>
  <strong>Date:</strong> 2024-10-30<br>
  <strong>Affiliation:</strong> Carnegie Mellon University, Cohen-Karni Group  || Neuromechatronics Lab
</p>

---

## 📜 Project Overview

(to populate) 

- **Objective:** (to populate)
- **Scope:** (to populate)
- **Libraries:** `Python`, `Pandas`, `SciPy`

---

## 📊 Notebook Outline

1. **Data Import & Preprocessing**


---

## 📚 References & Additional Resources

- [Github: Kilosort4](https://github.com/MouseLand/Kilosort/tree/main)
- [SpikeInterface](https://github.com/SpikeInterface)

---

<!-- Notebook Header End -->


# Libraries

In [ ]:
# Imports
from pathlib import Path
from tqdm import tqdm
import numpy as np

from probeinterface import write_prb
# Download channel maps for default probes
from kilosort.utils import download_probes
download_probes()

from spikeinterface.extractors import read_intan

from kilosort import run_kilosort
from kilosort import io

# File Definitions

In [2]:
# Parameters
RAT_NAME = 'DW322'
STIMULATION = 'DRGS_2_240918_130024'
DATA_DIRECTORY = Path(fr'D:\Data\CMU.80 Data\88 Analyzed Data\88.001 Kilosort on DW322')  # NOTE Specify the path where the data will be copied to, and where Kilosort4 results will be saved.
# Create path if it doesn't exist
DATA_DIRECTORY.mkdir(parents=True, exist_ok=True)

# Data Import

### defining filepath for data

In [3]:
# NOTE: You will need to select the appropriate data stream. If you run without
#       specifying `stream_id`, you will get an error message explaining what
#       each stream corresponds to.
filepath = Path(f"D:\Data\CMU.80 Data\82 External Data\82.002 Sample Rat Data from RM1 Project\DW322\{RAT_NAME}\{STIMULATION}\{STIMULATION}.rhd")
recording = read_intan(filepath, stream_id='0')

no probe information is given with this data, so we will need to define it or assign one from the probe interface library.

I will first show how to create the binary file using `Spike Interface` and numpy's `memmap()`

## Import data and probe using `SpikeInterface`

In [4]:
# NOTE: Data will be saved as np.int16 by default since that is the standard
#       for ephys data. If you need a different data type for whatever reason
#       such as `np.uint16`, be sure to update this.
dtype = np.int16
filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(
    recording, DATA_DIRECTORY, data_name=f'{RAT_NAME}_{STIMULATION}_data.bin', dtype=dtype,
    chunksize=60000, export_probe=True, probe_name='probe.prb'
    )

this code will complete, but because there is no probe associated with the data, I will need to define one later on

## Import data and probe using `Numpy memmap`

### defining ouput file and loading data into binary in chunks:

In [9]:
# Define output binary file
output_file = DATA_DIRECTORY / f'{RAT_NAME}_{STIMULATION}_data_memmap.bin'

# Get data shape
num_channels = recording.get_num_channels()
num_samples = recording.get_num_frames()

# Create a memory-mapped file without using `with`
data_dtype = np.int16  # This is typically the standard for electrophysiology data
memmap_file = np.memmap(output_file, dtype=data_dtype, mode='w+', shape=(num_samples, num_channels))

# Define the chunk size for processing (e.g., 60,000 samples at a time)
chunk_size = 60000

# Loop through the data and write to the binary file in chunks
for start_idx in range(0, num_samples, chunk_size):
    end_idx = min(start_idx + chunk_size, num_samples)
    
    # Extract data chunk from the RecordingExtractor
    data_chunk = recording.get_traces(start_frame=start_idx, end_frame=end_idx)
    
    # Write the chunk to the memory-mapped file
    memmap_file[start_idx:end_idx, :] = data_chunk.astype(data_dtype)

# Explicitly flush changes to disk
memmap_file.flush()

print(f"Data successfully saved to {output_file}")


Data successfully saved to D:\Data\CMU.80 Data\88 Analyzed Data\88.001 Kilosort on DW322\DW322_DRGS_2_240918_130024_data_memmap.bin


### exporting probe data

this will NOT run correctly, since there is no probe data

In [ ]:

# Get the probe from the recording (if available)
probe = recording.get_probe()

# Define the path to save the probe configuration
probe_path = DATA_DIRECTORY / 'probe.prb'

# Save the probe configuration
write_prb(probe_path, probe)


ValueError: There is no Probe attached to this recording. Use set_probe(...) to attach one.

## Defining a Probe